In [ ]:
import json
from pathlib import Path

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from tqdm.notebook import tqdm

In [ ]:
# meta
in_path = Path("/Users/kuangdai/Downloads/anim")
with open(in_path / "02_stations/args.json", "r") as fs:
    meta = json.load(fs)
with open(in_path / "02_stations/args_anim.json", "r") as fs:
    meta_anim = json.load(fs)

ulvz_dist = (meta["ulvz_dist_near"] + meta["ulvz_dist_far"]) / 2
ulvz_radius = meta_anim["ulvz_actual_radius"]
ulvz_height = meta_anim["ulvz_height"]

In [ ]:
# slide coordinates
grid_depth_solid = np.loadtxt(in_path / f"02_stations/grid_depth_anim_side_solid.txt")
grid_depth_fluid = np.loadtxt(in_path / f"02_stations/grid_depth_anim_side_fluid.txt")
grid_dist_anim_side = np.loadtxt(in_path / f"02_stations/grid_dist_anim_side.txt")
grid_dist_anim_both = np.concatenate((-grid_dist_anim_side[::-1], grid_dist_anim_side))
grid_dist_anim_shift = grid_dist_anim_both + ulvz_dist
r_solid, theta_solid = np.meshgrid(6371 - grid_depth_solid, grid_dist_anim_shift, indexing="ij")
x_solid = r_solid * np.sin(theta_solid)
z_solid = r_solid * np.cos(theta_solid)
r_fluid, theta_fluid = np.meshgrid(6371 - grid_depth_fluid, grid_dist_anim_shift, indexing="ij")
x_fluid = r_fluid * np.sin(theta_fluid)
z_fluid = r_fluid * np.cos(theta_fluid)

# top coordinates
grid_dist_anim = np.loadtxt(in_path / f"02_stations/grid_dist_anim_top.txt")
grid_azim_anim = np.loadtxt(in_path / f"02_stations/grid_azim_anim_top.txt")
grid_azim_anim = np.append(grid_azim_anim, grid_azim_anim[0])
theta, phi = np.meshgrid(grid_dist_anim, grid_azim_anim, indexing="ij")
x_top = np.sin(theta) * np.cos(phi)
y_top = np.sin(theta) * np.sin(phi)

In [ ]:
# specify output path
wave_path = Path("./outputs")
(wave_path / "png").mkdir(exist_ok=True)

# side wave
u_side_fluid = np.load(f"{wave_path}/outgoing_side_fluid.npz")["arr_0"]
u_side_solid = np.load(f"{wave_path}/outgoing_side_solid.npz")["arr_0"]
u_side_solid_both = np.concatenate((u_side_solid[..., ::-1, 0], u_side_solid[..., 1]), axis=-1)
u_side_fluid_both = np.concatenate((u_side_fluid[..., ::-1, 0], u_side_fluid[..., 1]), axis=-1)

# top wave
u_top_fluid = np.load(f"{wave_path}/outgoing_top_fluid.npz")["arr_0"]
u_top_solid = np.load(f"{wave_path}/outgoing_top_solid.npz")["arr_0"]
u_top_fluid = np.concatenate((u_top_fluid, u_top_fluid[..., 0:1]), axis=-1)
u_top_solid = np.concatenate((u_top_solid, u_top_solid[..., 0:1]), axis=-1)

# time
times = np.loadtxt(f"{wave_path}/outgoing_side_fluid_times.txt")

In [ ]:
# specify channel and time interval
channel = 1
t_interval = 1

# specify styles
v_min = u_side_solid_both.min() / 5
v_max = u_side_solid_both.max() / 5
cmap = "berlin"
outline_width = 0.3

# plot
for i_t, t in enumerate(tqdm(times)):
    if i_t % t_interval != 0:
        continue

    fig = plt.figure(dpi=300, figsize=(16 / 1.8, 8 / 1.8))
    gs = GridSpec(2, 2, wspace=-0.5, hspace=0.2)
    ax0 = fig.add_subplot(gs[:, 0])
    ax1 = fig.add_subplot(gs[0, 1], projection='polar')
    ax2 = fig.add_subplot(gs[1, 1], projection='polar')
    fig.patch.set_facecolor('dimgray')
    ax0.set_aspect(1.)
    ax1.set_aspect(1.)
    ax2.set_aspect(1.)
    ax0.axis("off")
    ax1.axis("off")
    ax2.axis("off")

    # plot data
    ax0.pcolormesh(x_solid, z_solid,
                   u_side_solid_both[i_t, channel, :, :], shading='gouraud', cmap=cmap,
                   vmin=v_min, vmax=v_max)
    ax0.pcolormesh(x_fluid, z_fluid,
                   u_side_fluid_both[i_t, channel, :, :], shading='gouraud', cmap=cmap,
                   vmin=v_min, vmax=v_max)
    ax1.pcolormesh(phi, theta,
                   u_top_solid[i_t, channel, 0, :, :], shading='gouraud', cmap=cmap,
                   vmin=v_min, vmax=v_max)
    ax2.pcolormesh(phi, theta,
                   u_top_fluid[i_t, channel, 0, :, :], shading='gouraud', cmap=cmap,
                   vmin=v_min, vmax=v_max)

    # add ulvz outline
    ax0.plot([3480 * np.sin(ulvz_dist - ulvz_radius), (3480 + ulvz_height) * np.sin(ulvz_dist - ulvz_radius)],
             [3480 * np.cos(ulvz_dist - ulvz_radius), (3480 + ulvz_height) * np.cos(ulvz_dist - ulvz_radius)],
             color='w', linewidth=outline_width)
    ax0.plot([3480 * np.sin(ulvz_dist + ulvz_radius), (3480 + ulvz_height) * np.sin(ulvz_dist + ulvz_radius)],
             [3480 * np.cos(ulvz_dist + ulvz_radius), (3480 + ulvz_height) * np.cos(ulvz_dist + ulvz_radius)],
             color='w', linewidth=outline_width)
    arc = patches.Arc((0, 0), width=(3480 + ulvz_height) * 2, height=(3480 + ulvz_height) * 2, angle=0,
                      theta1=90 - np.rad2deg(ulvz_dist + ulvz_radius),
                      theta2=90 - np.rad2deg(ulvz_dist - ulvz_radius),
                      color='w', linewidth=outline_width)
    ax0.add_patch(arc)
    arc = patches.Arc((0, 0), width=3480 * 2, height=3480 * 2, angle=0,
                      theta1=90 - np.rad2deg(grid_dist_anim_shift[-1]),
                      theta2=90 - np.rad2deg(grid_dist_anim_shift[0]),
                      color='w', linewidth=outline_width)
    ax0.add_patch(arc)
    circ = patches.Circle((0, 0), np.sin(ulvz_radius) * 1.02, color='w', linewidth=outline_width,
                          transform=ax1.transData._b, fill=False)
    ax1.add_patch(circ)
    circ = patches.Circle((0, 0), np.sin(ulvz_radius) * 1.02, color='w', linewidth=outline_width,
                          transform=ax2.transData._b, fill=False)
    ax2.add_patch(circ)

    # texts
    ax0.set_title("Central Slice", fontsize=8, color="w")
    ax1.set_title("CMB Solid", fontsize=8, color="w")
    ax2.set_title("CMB Fluid", fontsize=8, color="w")
    w0 = -0.8
    h0 = 0.65
    delta = 0.06
    ax0.text(w0, h0 - delta * 0, "ULVZ Properties:", transform=ax0.transAxes, fontsize=8, va='top', c="w")
    ax0.text(w0, h0 - delta * 1, "    Distance to Event = 61 deg", transform=ax0.transAxes, fontsize=8, va='top',
             c="w")
    ax0.text(w0, h0 - delta * 2, "    Height = 20 km", transform=ax0.transAxes, fontsize=8, va='top', c="w")
    ax0.text(w0, h0 - delta * 3, "    Radius = 5 deg", transform=ax0.transAxes, fontsize=8, va='top', c="w")
    ax0.text(w0, h0 - delta * 4, "    Vs Reduction = 30%", transform=ax0.transAxes, fontsize=8, va='top', c="w")
    ax0.text(w0, h0 - delta * 5, "Simulation Period = 4 s", transform=ax0.transAxes, fontsize=8, va='top', c="w")
    ax0.text(w0, h0 - delta * 6, "Phase Shown = Sdiff", transform=ax0.transAxes, fontsize=8, va='top', c="w")
    ax0.text(w0, h0 - delta * 7, "Channel Shown = Transverse", transform=ax0.transAxes, fontsize=8, va='top', c="w")
    ax0.text(w0 - 0.05, 1.05, "AxiSEM3D", transform=ax0.transAxes, fontsize=12,
             va='top', c="w", family='Times New Roman', weight='bold')
    # time
    ax0.text(w0, 0.1, f"t = {times[i_t]:.2f} s",
             transform=ax0.transAxes, fontsize=12, va='top', c="w")
    plt.savefig(wave_path / f"png/{i_t:04d}.png", bbox_inches='tight', pad_inches=0.2)
    plt.close()